In [2]:
import sys
sys.version

'3.9.9 (main, Nov 17 2021, 16:30:15) \n[GCC 10.2.1 20210110]'

In [12]:
import urllib3
import requests
import json
from base64 import b64encode
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [13]:
#########################
# Cluster情報を入力
#########################

#POC20
prism_ip = '10.149.20.41'
prism_user = 'admin'
prism_pass = 'Nutanix/4u123!'

In [14]:
# authentication
# 参考 https://www.nutanix.dev/reference/prism_central/v3/authentication
request_url = 'https://' + prism_ip + ':9440/api/nutanix/v3/vms/list'

encoded_credentials = b64encode(bytes(f'{prism_user}:{prism_pass}', encoding='ascii')).decode('ascii')
auth_header = f'Basic {encoded_credentials}'
headers = {'Accept': 'application/json', 'Content-Type': 'application/json', 'Authorization': f'{auth_header}', 'cache-control': 'no-cache'}

In [15]:
payload = '{"kind":"vm", "length":256}'

In [16]:
# 200が返ってこれば OK
# 取得失敗の処理
try:
    response = requests.request('post', request_url, data=payload, headers=headers, verify=False, timeout=3.5)

except requests.exceptions.ConnectTimeout:
    response = "Timeout shimasita"

response

<Response [200]>

In [17]:
# json整形
res_vms = response.json()

# なんとなく確認、今後ブラウザ表示とかに使う
res_vms['metadata']

{'total_matches': 29, 'kind': 'vm', 'length': 29, 'offset': 0}

In [18]:



#ここからElasticsearch操作

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from datetime import datetime

es = Elasticsearch('elasticsearch:9200')

indices = es.cat.indices(index='*', h='index').splitlines()

# Warningは一旦無視
# index一覧を表示
indices

/usr/local/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


['.geoip_databases',
 '.apm-custom-link',
 'nutanix-volume_group',
 '.apm-agent-configuration',
 '.kibana_task_manager_7.14.2_001',
 '.kibana_7.14.2_001',
 'vm_list',
 '.async-search',
 'volume_group',
 '.tasks',
 'nutanix-vm_list',
 '.kibana-event-log-7.14.2-000001']

In [19]:
# 現状の index確認にして、存在してなければ作成
index_name = 'vm_list'

if index_name not in indices:
    es.indices.create(index=index_name)

In [20]:
cluster_name = res_vms['entities'][0]['spec']['cluster_reference']['name']
timestamp = datetime.utcnow()
print(cluster_name)

POC20


In [21]:
# REST API取得結果から entitiesだけを抜き出して、配列で格納
# VMごとの _docになり検索しやすい
actions = []
for entity in res_vms['entities']:
    entity['timestamp'] = timestamp
    entity['cluster_name'] = cluster_name
    actions.append({'_index':index_name, '_source':entity}) 

In [23]:
# Elastcisearchへ投入 ※Waringは無視してOK
reaction = helpers.bulk(es, actions)


In [25]:
reaction, len(actions)

((29, []), 29)

29

### このあと5分くらい待ってから、Kibanaの画面を確認しにいく  

[localhost:5601](localhost:5601)  



## volume group も取得して、Elasticsearchへ

In [18]:
# get volume_group
request_url = 'https://' + prism_ip + ':9440/PrismGateway/services/rest/v2.0/volume_groups/'

# 200が返ってこれば OK
response = requests.request('get', request_url, headers=headers, verify=False)
response.status_code

200

In [20]:
volume_group = response.json()

In [24]:
len(volume_group['entities'])

26

In [25]:
# 現状の index確認にして、存在してなければ作成
index_name = 'volume_group'

indices = es.cat.indices(index='*', h='index').splitlines()
if index_name not in indices:
    es.indices.create(index=index_name)

# Elasticsearch投入用に整形
actions = []
for entity in volume_group['entities']:
    entity['timestamp'] = timestamp
    entity['cluster_name'] = cluster_name
    actions.append({'_index':index_name, '_source':entity}) 

In [26]:
# Elasticsearchへ投入
helpers.bulk(es, actions)

(26, [])

### データセット

In [ ]:
def set_data():
    title = "Time select Complete"
    cluster_name = request.form['cluster_name']
    timestamp_utcstr = request.form['timestamp-utcstr']

    timeslot = es.get_timeslot(cluster_name)
    timestamp_list =  data_broker.change_timestamp_list(timestamp_utcstr)

## Elasticsearchから取得

In [79]:
def change_timeslot_list(timeslot):
    timeslot_dict = []
    for oneslot in timeslot:
        _utc = re.split('[T.]', oneslot)
        utc = _utc[0] + " " + _utc[1]
        utc_time = datetime.strptime(utc, '%Y-%m-%d %H:%M:%S')
        _jst_time = utc_time.astimezone(timezone(timedelta(hours=+9)))
        jst_time = datetime.strftime(_jst_time, '%Y-%m-%d %H:%M:%S')

        timeslot_dict.append({'value_time':oneslot, 'local_time':jst_time})
    return timeslot_dict

def change_timestamp_list(timestamp):
    timestamp_dict = []
    _utc = re.split('[T.]', timestamp)
    utc = _utc[0] + " " + _utc[1]
    utc_time = datetime.strptime(utc, '%Y-%m-%d %H:%M:%S')
    _jst_time = utc_time.astimezone(timezone(timedelta(hours=+9)))
    jst_time = datetime.strftime(_jst_time, '%Y-%m-%d %H:%M:%S')

    timestamp_dict.append({'value_time':timestamp, 'local_time':jst_time})
    return timestamp_dict

class Ela():
    def __init__(self, es_server):
        self.es = Elasticsearch(es_server)

    def get_indices(self):
        es = self.es
        return es.cat.indices()

    # insert vm_list
    def vm_list(self, r_json, timestamp):
        index_name = 'vm_list'
        cluster_name = r_json['entities'][0]['spec']['cluster_reference']['name']

        actions = []
        for entity in r_json['entities']:
            entity['timestamp'] = timestamp
            entity['cluster_name'] = cluster_name
            actions.append({'_index':index_name, '_source':entity})

        return actions, cluster_name

    # insert volume group
    def volume_group(self, vg_json, timestamp, cluster_name):
        es = self.es
        index_name = 'volume_group'

        actions = []
        for entity in vg_json['entities']:
            entity['timestamp'] = timestamp
            entity['cluster_name'] = cluster_name
            actions.append({'_index':index_name, '_source':entity})
        helpers.bulk(es, actions)

    def input_data(self, response, res_list):
        es = self.es
        r_json = response.json()

        # vm_list
        timestamp = datetime.utcnow()
        actions, cluster_name = self.vm_list(r_json, timestamp)
        reaction = helpers.bulk(es, actions)

        # volume_group
        vg_json = res_list['volume_group'].json()
        self.volume_group(vg_json, timestamp, cluster_name)

        return reaction[0], cluster_name

    def get_eslist(self, index_name, timestamp, cluster_name):
        es = self.es
        query =  {
            "function_score" : {
                "query": { "bool": { "must": [
                    {"match": { "timestamp" : timestamp}},
                    {"match": { "cluster_name" : cluster_name}}
                ]}}
            }
        }
        res = es.search(index=index_name, query=query, size=512)
        return [s['_source'] for s in res['hits']['hits'] ]
    
    def search_eslist(self, index_name, timestamp, cluster_name, keyword):
        es = self.es
        query =  {
            "function_score" : {
                "query": { "bool": { "must": [
                    {"match": { "cluster_name": cluster_name}},
                    {"match": { "timestamp": timestamp}},
                    {"multi_match": { 
                      "query": keyword,
                      "fields": ["metadata.uuid",  "attachment_list.vm_uuid", "uuid"]
                    }}
                ]}}
            }
        }
        res = es.search(index=index_name, query=query, size=512)
        return [s['_source'] for s in res['hits']['hits'] ]
    
    
    def get_timeslot(self, cluster_name):
        es = self.es
        index_name = 'vm_list'

        query = {
            "function_score" : {
                "query": {"match": { 'status.cluster_reference.name' : cluster_name}}
            }
        }
        aggs =  {
            "group_by_timestamp": {"terms": { "field" : "timestamp", "size" : 1000}}
        }
        res = es.search(index=index_name, query=query, aggs=aggs)
        _timeslot = [slot['key_as_string'] for slot in res['aggregations']['group_by_timestamp']['buckets']]
        timeslot = sorted(_timeslot, reverse=True)
        timeslot_dict = change_timeslot_list(timeslot)
        return timeslot_dict


In [80]:
ELASTIC_SERVER = 'elasticsearch:9200'
es = Ela(es_server=ELASTIC_SERVER)

cluster_name = 'POC20'
#index_name = 'vm_list'
index_name = 'volume_group'
timeslot = es.get_timeslot(cluster_name)
timestamp_utcstr = timeslot[0]['value_time']
timestamp_utcstr

'2021-12-05T16:19:41.064Z'

In [49]:
info = es.get_eslist(index_name, timestamp_utcstr, cluster_name)
len(info)

27

In [50]:
import json 
with open('flaskr/sample/'+ index_name +'.json', mode='w') as f:
    f.write(json.dumps(info))


In [82]:
keyword = 'dbb56a2e-8ef0-45a8-b4eb-b5af64d6241f'
index_name = ['vm_list', 'volume_group']
info = es.search_eslist(index_name, timestamp_utcstr, cluster_name, keyword)
len(info)

1

In [75]:
info

[{'uuid': 'dbb56a2e-8ef0-45a8-b4eb-b5af64d6241f',
  'name': 'File_Analytics_VG',
  'description': '',
  'logical_timestamp': 3,
  'disk_list': [{'index': 0,
    'vmdisk_uuid': '6dc80455-c895-47ed-8278-dd4d94a1e912',
    'container_id': 12,
    'storage_container_id': 12,
    'container_uuid': '0f887a94-6051-4996-8759-48f62188c175',
    'storage_container_uuid': '0f887a94-6051-4996-8759-48f62188c175',
    'vmdisk_size_mb': 2097152,
    'vmdisk_size_bytes': 2199023255552,
    'vmdisk_path': '/default-container-90129684887843/.acropolis/vmdisk/6dc80455-c895-47ed-8278-dd4d94a1e912',
    'flash_mode_enabled': False}],
  'iscsi_target': 'fa-dbb56a2e-8ef0-45a8-b4eb-b5af64d6241f',
  'attachment_list': [{'vm_uuid': '136b24a0-8c2d-4fe1-9ef5-10bce0299f23'}],
  'flash_mode_enabled': False,
  'is_shared': True,
  'timestamp': '2021-12-05T16:19:41.064491',
  'cluster_name': 'POC20'}]

In [66]:
import json 
with open('flaskr/sample/result.json', mode='w') as f:
    f.write(json.dumps(info))


# 実験場

In [2]:
def get_timeslot(index_name, field, cluster_name):
    query = {
        "function_score" : {
            "query": {"match": { field : cluster_name}}
        }
    }

    aggs = { 
        "group_by_timestamp": {"terms": { 
            "field" : "timestamp",
            "size" : 10000
        }}
    }
    
    res = es.search(index=index_name, query=query, aggs=aggs)
    return res['aggregations']

In [3]:
groupby_time = get_timeslot('vm_list', 'status.cluster_reference.name', 'POC20')

In [4]:
timeslot = [slot['key_as_string'] for slot in groupby_time['group_by_timestamp']['buckets']]
timeslot.sort(reverse=True)

In [5]:
timeslot

['2021-12-03T00:06:29.741Z',
 '2021-12-02T15:00:33.886Z',
 '2021-11-26T09:00:18.783Z',
 '2021-11-26T00:15:26.759Z',
 '2021-11-26T00:13:59.363Z',
 '2021-11-26T00:09:27.177Z',
 '2021-11-25T23:32:18.882Z',
 '2021-11-25T13:38:20.455Z',
 '2021-11-25T13:14:26.568Z',
 '2021-11-25T13:10:44.727Z',
 '2021-11-25T13:03:09.212Z',
 '2021-11-25T12:59:10.242Z',
 '2021-11-25T12:56:55.235Z',
 '2021-11-25T12:52:58.863Z',
 '2021-11-25T11:50:05.969Z',
 '2021-11-25T11:40:58.369Z',
 '2021-11-25T11:39:10.954Z',
 '2021-11-25T11:37:09.891Z',
 '2021-11-25T11:31:31.655Z',
 '2021-11-25T11:28:04.006Z',
 '2021-11-25T11:27:32.138Z',
 '2021-11-25T11:24:45.933Z',
 '2021-11-25T11:19:37.010Z',
 '2021-11-25T09:16:21.059Z',
 '2021-11-25T08:47:29.370Z',
 '2021-11-24T11:28:33.911Z',
 '2021-11-24T10:24:34.348Z',
 '2021-11-24T10:23:51.714Z',
 '2021-11-24T10:03:59.508Z',
 '2021-11-24T10:02:06.266Z',
 '2021-11-24T09:59:59.892Z',
 '2021-11-24T09:07:59.305Z',
 '2021-11-23T15:43:42.647Z',
 '2021-11-23T14:03:36.100Z',
 '2021-11-23T1

In [6]:
from datetime import datetime, timezone, timedelta
import re

In [7]:
def change_timeslot_list(timeslot):
    timeslot_dict = []
    for oneslot in timeslot:
        _utc = re.split('[T.]', oneslot)
        utc = _utc[0] + " " + _utc[1]
        utc_time = datetime.strptime(utc, '%Y-%m-%d %H:%M:%S')
        _jst_time = utc_time.astimezone(timezone(timedelta(hours=+9)))
        jst_time = datetime.strftime(_jst_time, '%Y-%m-%d %H:%M:%S')
        
        timeslot_dict.append({'value_time':oneslot, 'local_time':jst_time})
    return timeslot_dict

In [8]:
def change_timestamp_list(timestamp):
    timestamp_dict = []

    _utc = re.split('[T.]', timestamp)
    utc = _utc[0] + " " + _utc[1]
    utc_time = datetime.strptime(utc, '%Y-%m-%d %H:%M:%S')
    _jst_time = utc_time.astimezone(timezone(timedelta(hours=+9)))
    jst_time = datetime.strftime(_jst_time, '%Y-%m-%d %H:%M:%S')

    timestamp_dict.append({'value_time':timestamp, 'local_time':jst_time})
    return timestamp_dict

In [9]:
timestamp_dict = change_timestamp_list(timeslot[0])
timestamp_dict

[{'value_time': '2021-12-03T00:06:29.741Z',
  'local_time': '2021-12-03 09:06:29'}]

In [11]:
timeslot_dict = change_timeslot_list(timeslot)
timeslot_dict[0]

{'value_time': '2021-12-03T00:06:29.741Z', 'local_time': '2021-12-03 09:06:29'}

In [ ]:
def get_

In [122]:
def get_doc(index_name, field, keyword):
    query =  { 
        "function_score" : {
            "query": {"match": { field : keyword}}
        }
    }
    res = es.search(index=index_name, query=query, size=512)
    return [s['_source'] for s in res['hits']['hits'] ]

In [123]:
field = "timestamp"
aa = get_doc('vm_list', field, timestamp)

In [124]:
len(aa), [a['spec']['name'] for a in aa]

(29,
 ['PC-POC20',
  'win2012',
  'FoundationVM5.0.4',
  'PC-POC20',
  'PC-NameOption-2',
  'PC-NameOption-3',
  'pc-1',
  'pc-2',
  'pc-3',
  'pc-01',
  'pc-03',
  'pc-02',
  'otani-vm1',
  'otani-vm2',
  'otani-vm3',
  'ebgp',
  'Nutanix-Clone-ESXi6.7',
  'NTNX-ohtani01-1',
  'NTNX-FA-ohtani-fa',
  'NTNX-ohtani02-1',
  'NTNX-ohtani02-2',
  'NTNX-ohtani02-3',
  'foundation-vm-5.0.3',
  'Win10yuki',
  'CentOS8',
  'Nutanix-Clone-CentOS8',
  'LGWAN_VM',
  'move4.2.1',
  'move4.0'])

In [31]:
#from requests.exceptions import Timeout

class NutanixAPI():
    def get_vmlist(self, prism_ip, prism_user, prism_pass):
        #########################
        # Cluster情報を入力
        #########################
        cluster_name = "POC20"
        #prism_ip = '10.149.20.41'
        #prism_user = 'dmin'
        #prism_pass = 'Nutanix/4u123!'

        # authentication
        # 参考 https://www.nutanix.dev/reference/prism_central/v3/authentication
        request_url = 'https://' + prism_ip + ':9440/api/nutanix/v3/vms/list'

        encoded_credentials = b64encode(bytes(f'{prism_user}:{prism_pass}', encoding='ascii')).decode('ascii')
        auth_header = f'Basic {encoded_credentials}'
        payload = '{"kind":"vm", "length":256}'
        headers = {'Accept': 'application/json', 'Content-Type': 'application/json', 'Authorization': f'{auth_header}', 'cache-control': 'no-cache'}

        # 200が返ってこれば OK
        try:
            response = requests.request('post', request_url, data=payload, headers=headers, verify=False, timeout=3.5)
        
        except requests.exceptions.ConnectTimeout:
            response = "Timeout shimasita"
        
        return response

In [32]:
ntnx = NutanixAPI()

In [35]:
ip = '10.149.20.41'
user = 'admin'
passs = 'Nutanix/4u123!'
response = ntnx.get_vmlist(ip, user, passs)

In [36]:
response

<Response [200]>

In [28]:
if hasattr(aa, 'status_code'):
    if response.status_code == 200:
        print("OK")
    else:
        r_json = response.json()
        print(r_json['message_list'][0]['message'])
else:
    print(aa)

{'reason': 'AUTHENTICATION_REQUIRED', 'message': 'Authentication required.', 'details': 'Basic realm="Intent Gateway Login Required"'}


In [98]:
abc = ''
if 'abc' in locals(): print('yoshi')

yoshi
